# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: The only features that are continous are 'age' and 'absences'. If a regression model were to be used, these two features would be the only input for predicting passed 'yes' or 'no'.  Excluding the target feature 'passed', there are a combination of 30 features that are categorical, binary,and the two previously mentioned continous features. And the final point, to look at the features that help determine the target feature, student success of passing or failing. This is an obvious case for classification.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# Calculate number of students
n_students = student_data.shape[0]

# Calculate number of features
n_features = student_data.shape[1] - 1  #Subtracting 1 from features to exclude "passed" target feature

# Calculate passing students
n_passed = student_data[student_data.passed == 'yes'].shape[0]

# Calculate failing students
n_failed = student_data[student_data.passed == 'no'].shape[0]

# Calculate graduation rate
grad_rate = 100.0*n_passed / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train
test_prop = 1.0 * num_test / (num_test + num_train) 

# Shuffle and split the dataset into the number of training and testing points above
# On your example you had "random_state=42", I tested multiple values and arrived on 20 to get the 'yes' pct to your % below.
# What is the random state doing/changing?
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, random_state=20, test_size=test_prop)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print "Train set 'yes' pct = {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Test  set 'yes' pct = {:.2f}%".format(100 * (y_test == 'yes').mean())

Training set has 300 samples.
Testing set has 95 samples.
Train set 'yes' pct = 67.00%
Test  set 'yes' pct = 67.37%


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: 
Support Vector Machines (SVM)

•	Describe one real-world application in industry where the model can be applied
Classification of images can also be performed using SVMs. Experimental results show that SVMs achieve significantly higher search accuracy than traditional query refinement schemes after just three to four rounds of relevance feedback. This is also true of image segmentation systems, including those using a modified version SVM that uses the privileged approach as suggested by Vapnik.
[1] https://en.wikipedia.org/wiki/Support_vector_machine#Applications

•	What are the strengths of the model; when does it perform well?
There are four main advantages: First, it has a regularization parameter, which makes the user think about avoiding over-fitting. Second, it uses the kernel trick, so one can build in expert knowledge about the problem via engineering the kernel. Third, an SVM is defined by a convex optimization problem (no local minima) for which there are efficient methods (e.g. SMO). Last, it approximates a bound on the test error rate, and there is a substantial body of theory behind it which suggests it should be a good idea.
[2] http://stats.stackexchange.com/questions/24437/advantages-and-disadvantages-of-svm

•	What are the weaknesses of the model; when does it perform poorly?
Potential drawbacks of the SVM include the following aspects:
•	Requires full labeling of input data
•	Uncalibrated class membership probabilities
•	The SVM is only directly applicable for two-class tasks. Therefore, algorithms that reduce the multi-class task to several binary problems must be applied. 

•	Parameters of a solved model are difficult to interpret.
Drawback of the SVMs, one must choose the kernel. That means you must provide the true structure of the data as an input, while other algorithms, like neural networks or random-forests, try to automatically find the structure. Also, you have to tune the parameters for the kernels and the C parameter which can be time consuming and decrease the performance if done wrong. Essentially, SVMs tend to run slower than other algorithms and perform poorly on highly skewed/imbalanced data sets. 
[3] https://en.wikipedia.org/wiki/Support_vector_machine

•	What makes this model a good candidate for the problem, given what you know about the data?
The data has been processed into binary format with conversion of “yes” and “no” features to numeric values and dummy variables for categorical features. And the target variable is requisitely binary. The size of the feature space is not overly large in comparison to the sample space. And the data doesn’t exhibit skewness or imbalance.

Adaboost
•	Describe one real-world application in industry where the model can be applied.  
AdaBoost (Adaptive Boosting) is a powerful classifier that works well on both basic and more complex recognition problems, such as biology, computer vision, and speech processing. [4] http://www.nickgillian.com/wiki/pmwiki.php/GRT/AdaBoost 

What are the strengths of the model; when does it perform well?
Evaluating every feature can reduce not only the speed of classifier training and execution, but in fact reduce predictive power, per the Hughes Effect. Unlike neural networks and SVMs, the AdaBoost training process selects only those features known to improve the predictive power of the model, reducing dimensionality and potentially improving execution time as irrelevant features do not need to be computed. [5] http://www.nickgillian.com/wiki/pmwiki.php/GRT/AdaBoost  
What are the weaknesses of the model; when does it perform poorly?
Weak classifiers too weak can lead to low margins, and can also lead to overfitting. AdaBoost can be sensitive to noisy data, outliers and particularly uniform noise. [6] http://math.mit.edu/~rothvoss/18.304.3PM/Presentations/1-Eric-Boosting304FinalRpdf.pdf 
•	What makes this model a good candidate for the problem, given what you know about the data?
There are no outliers or uniform noise. And the data is apparently non-linear, Adaboost works well in the above case of classification.

LogisticRegression
•	Describe one real-world application in industry where the model can be applied. 
Logistic regression may be used to predict whether a patient has a given disease (e.g. diabetes; coronary heart disease), based on observed characteristics of the patient (age, sex, body mass index, results of various blood tests, etc.) [7] https://en.wikipedia.org/wiki/Logistic_regression 
•	 What are the strengths of the model; when does it perform well?
It’s robust: the independent variables don’t have to be normally distributed, or have equal variance. It does not assume a linear relationship between the features and target. Can handle skewness of data, and no assumption of homogeneity of variance assumption. Performs well in Bernoulli situations, i.e. Probability of passing an exam versus hours of study (similar to our data set.) [8] https://en.wikipedia.org/wiki/Logistic_regression 

•	What are the weaknesses of the model; when does it perform poorly?
The advantages of logistic regression come at a cost: it requires much more data to achieve stable, meaningful results. For logistic regression, at least 50 data points per predictor is necessary to achieve stable results. [9] https://victorfang.wordpress.com/2011/05/10/advantages-and-disadvantages-of-logistic-regression/ 

•	What makes this model a good candidate for the problem, given what you know about the data?
Ok, so after choosing and coding each of the algorithms and reaching my conclusions, I found the above noted weakness. The LogisticRegression (LR) model performed reasonable well in comparison to the other two, and I have used it in the past, but I would not choose it again. Strictly following our directions and what we learned in the videos and quizzes, I didn’t see any reason to not choose LR, but once I’d researched it, I see that it really isn’t a good candidate given the number of features and sample size. It may begin to do better as larger datasets are analyzed and it has good runtime even for big datasets. But, given the present situation and hindsight I wouldn’t choose it for this scenario.



 **

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    print "------------------------------------------"
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))  

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

# Initialize the three models
clf_A = SVC(random_state=None)
clf_B = AdaBoostClassifier(random_state=None)
clf_C = LogisticRegression(random_state=None)
models = {"SVC": clf_A, "AdaBoost": clf_B, "Logistic Regression": clf_C}

# Set up the training set sizes
sample_100 = np.random.choice(300, 100, replace=False)
sample_200 = np.random.choice(300, 200, replace=False)

train_100 = (X_train.iloc[sample_100], y_train.iloc[sample_100])

train_200 = (X_train.iloc[sample_200], y_train.iloc[sample_200])

train_300 = (X_train, y_train)

training_sets = (train_100, train_200, train_300)

# Execute the 'train_predict' function for each classifier and each training set size

for model_name, model in models.items():
    print "\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n"
    print "Testing {}\n".format(model_name)
    for aset in training_sets:
        train_predict(model, aset[0], aset[1], X_test, y_test)


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Testing SVC

------------------------------------------
Training a SVC using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8627.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8158.
------------------------------------------
Training a SVC using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.8544.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8205.
------------------------------------------
Training a SVC using a training set size of 300. . .
Trained model in 0.0090 seconds
Made predictions in 0.0060 seconds.
F1 score for training set: 0.8565.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.8205.

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Testing Logistic Regression

------------------------------------------
Training a LogisticRegression usi

In [8]:
# Benchmark when predicting all 'yes'
print "F1 score for predicting all \"yes\" on test set: {:.4f}".format(
        f1_score(y_test, ['yes']*len(y_test), pos_label='yes', average='binary'))  

F1 score for predicting all "yes" on test set: 0.8050


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector CLassifier (SVC) **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0030            |       0.0020           |       0.8750    |       0.8212    |
| 200               |       0.0040            |       0.0010           |       0.8528    |       0.8050    |
| 300               |       0.0070            |       0.0020           |       0.8565    |       0.8205    |

** Classifer 2 - AdaBoost **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0730            |       0.0030           |       0.9844     |       0.7132    |
| 200               |       0.0760            |       0.0030           |       0.8844     |       0.7660    |
| 300               |       0.0850            |       0.0020           |       0.8396     |       0.8054    |

** Classifer 3 - Logistic Regression **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0010            |       0.0010           |       0.9091     |       0.7941    |
| 200               |       0.0020            |       0.0000           |       0.8618     |       0.7862    |
| 300               |       0.0020            |       0.0000           |       0.8311     |       0.8000    |

The values in the table above changed quite a bit. The train/test F1 values, as the set size increased, began
to converge as I would want/expect. Is this due to the stratification code I added above? I'm 
thinking yes, because that is the only difference from my previous notebook. 

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: 

In the comparison process I evaluated 3 classifier algorithms (Logistic Regression, AdaBoost, and SVM) over the sample dataset. Of the three, I believe the SVM model will perform best with the current size of the data set. The other two algorithms performed well in the training phases, but not as well in the testing portion. Meaning, going forward with future data sets, or student data, the SVM model will correctly choose the students needing further intervention with a higher accuracy. 

There is one caveat to consider, if the above dataset is representative of future datasets, in consideration of size, then the SVM model will work well going forward. However, if instead of hundreds or even several thousands of students, there are tens or hundreds of thousands of students to be run in any one dataset, then the SVM model may be costlier than one of the other models. In fact, I would even recommend the Logistic Regression model as being the more cost effective, as well as working better with very large datasets with consideration of computing time and accuracy of prediction. 

 **

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: 

The Support Vector Machine (SVM) takes data about previous students (age, gender, family, etc), and uses them to create a function that draws a boundary between the students who passed and those who didn't. The boundary should be drawn so as to maximize the distance between students who passed and those that failed. This ‘maximum margin’ will give the most buffer for future students that may have slightly different characteristics, to be categorized correctly as students in need of intervention and those don’t or may only need to be monitored. For instance, note the graph below and let the red squares be students that failed and the blue circles be students that passed. The Optimal hyperplane is drawn to maximize the margin. The squares and circles correspond to students and the fully colored in squares and circle respectively are the vectors (points in 2D) that give SVM its name. If in the future a student is plotted closer than either of the squares or circle, the margin gives a better chance these students will be categorized appropriately. This way, the cost of intervention can be kept down when the charateristics are different, but still classified correctly. 
**

![SVM](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1468014011/optimal-hyperplane.png)

Now, unlike the above case of two characteristics, the data set has 30. This is not a problem for SVM and as in the graph below, sometimes there is no easy line that separates the points. SVM can project the points in to a higher dimension and draw a hyperplane that then separates the blue circles (passed) and red circles (failed). Again, classifying the students correctly that need help from those that don’t. 

![SVM](https://udacity-github-sync-content.s3.amazonaws.com/_imgs/19273/1464624531/1gvce.png)

And finally, SVM can be used on an incoming freshman class, for instance. After all the data is gathered as in the dataset above, prepared by the program and run through the SVM to predict the students that will need intervention from those that don’t. Thereby saving on costs of the program when applied to the needy students, yet not letting them slip through the cracks. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import normalize
X_train = normalize(X_train)
X_test = normalize(X_test)

# Create the parameters list you wish to tune
C_range = np.logspace(-2, 2, 20)
gamma_range = np.logspace(-2, 2, 20)
parameters = dict(gamma=gamma_range, C=C_range)

# Initialize the classifier
clf = SVC()

# Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, greater_is_better=True, pos_label='yes')

# Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, cv=5, verbose=1, n_jobs=-1, pre_dispatch= '2*n_jobs', scoring='f1_micro')

# Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print clf
print "-----------------------------------------------"
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "-----------------------------------------------"
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1033 tasks      | elapsed:    5.3s


SVC(C=2.0691380811147901, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=2.0691380811147901,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
-----------------------------------------------
Made predictions in 0.0040 seconds.
Tuned model has a training F1 score of 0.8260.
-----------------------------------------------
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8205.


[Parallel(n_jobs=-1)]: Done 1985 out of 2000 | elapsed:    7.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    7.2s finished


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: 

The final model had an F1 score for training of 0.8260 and for testing 0.8205. Both of these numbers are comparable to the untuned model. Which is a bit disconcerting that the model wasn't able to be tuned better. However, it may perform better with future data, which is the important aspect of this project. Of the 130 students who failed, having this model applied, early enough to their data would have resulted in identifying 106 students that needed help. If that help would then have produced a "passed" score for the target variable, then 371 students would have "passed" and the graduation rate would have been 93.9%. The school districts goal of a 95% graduation rate would be that much closer. With some other criteria, teacher observations for instance, the goal could be reasonably reached. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.